## import package

In [ ]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 19.8 MB/s 
     |████████████████████████████████| 7.6 MB 54.1 MB/s 
     |████████████████████████████████| 182 kB 64.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda:0


In [ ]:
# Training data file
directory="/content/drive/MyDrive/Colab Notebooks/AIcup2022/"


file=directory+"train.csv"
df=pd.read_csv(file, encoding = "ISO-8859-1")

In [ ]:
file=directory+"test.csv"
df_test=pd.read_csv(file, encoding = "ISO-8859-1")

In [ ]:
df = df.drop(columns=['Unnamed: 6', 'total no.: 7987'])

In [ ]:
df_test

,id,q,r,s
0,6199,"""-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ...","""If so , why do we still have apes , and why a...",DISAGREE
1,5807,"""There 's a lot of discussion there on that is...","""Of course . The makers of Expelled were withi...",DISAGREE
2,8487,"""`` It 's not helping . The guns these people ...","""Oh , I would wager about like Mexico , about ...",DISAGREE
3,1760,"""Shooting : 3 seriously injured in Auburn shoo...","""Pickup strikes group of four youths | Houston...",AGREE
4,6228,"""This is the argument concerning 'choice ' tha...","""I believe there is a point at which we ( soci...",DISAGREE
...,...,...,...,...
2011,9499,"""You are betraying your belief system .""","""Yep . ( I 'm assuming that by `` belief syste...",AGREE
2012,4611,"""You are in a loud minority , railing against ...","""Being in the minority or in the majority is i...",DISAGREE
2013,9328,"""You bet your XXX that 'd make me happy .""","""Well , first , I probably would n't bet my XX...",DISAGREE
2014,5225,"""you say `` f * * * the Constitution. ``""","""and gun nuts say f * * * the children when we...",DISAGREE


In [ ]:
df

,id,q,r,s,q',r'
0,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True ."""
1,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""can go both ways . We all doubt . It is what ...","""True"""
2,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True"""
3,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""seem to support the killing of certain people""","""based on the idea that people are dispensible..."
4,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""you seem to support the killing of certain pe...","""based on the idea that people are dispensible"""
...,...,...,...,...,...,...
38341,10001,"""good thing this argument has never been done ...","""And teen sex does n't , by the very nature of...",DISAGREE,"""You are much better off making theft legal an...","""And teen sex does n't , by the very nature of..."
38342,10002,"""I know one thing , anything that happens , po...","""Was n't sinjin crowing about his plans to tak...",DISAGREE,"""I know one thing , anything that happens , po...","""Was n't sinjin crowing about his plans to tak..."
38343,10002,"""I know one thing , anything that happens , po...","""Was n't sinjin crowing about his plans to tak...",DISAGREE,"""FBI Arrests Three Men in Terror Plot that Tar...","""Was n't sinjin crowing about his plans to tak..."
38344,10003,"""I enjoy Botany more than most things and I ha...","""Hi Smallax , welcome to the forum . I did a s...",AGREE,"""I enjoy Botany more than most things and I ha...","""Hi Smallax , welcome to the forum . I did a s..."


## Data process

In [ ]:
# train
df[['q','r',"q'","r'"]] = df[['q','r',"q'","r'"]].apply(lambda x: x.str.strip('\"'))

In [ ]:
# test
df_test[['q','r']] = df_test[['q','r']].apply(lambda x: x.str.strip('\"'))

In [ ]:
# train
df['r'] = df['s'] + ':' + df['r']

In [ ]:
# test
df_test['r'] = df_test['s'] + ':' + df_test['r']

In [ ]:
# train
df['sub_q_true'] = [1 if x in y else 0 for x,y in zip(df["q'"],df["q"])]
df['sub_r_true'] = [1 if x in y else 0 for x,y in zip(df["r'"],df["r"])]
df['sub_both'] = df['sub_q_true']*df['sub_r_true']

In [ ]:
df

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1
...,...,...,...,...,...,...,...,...,...
38341,10001,good thing this argument has never been done !...,"DISAGREE:And teen sex does n't , by the very n...",DISAGREE,You are much better off making theft legal and...,"And teen sex does n't , by the very nature of ...",1,1,1
38342,10002,"I know one thing , anything that happens , pol...",DISAGREE:Was n't sinjin crowing about his plan...,DISAGREE,"I know one thing , anything that happens , pol...",Was n't sinjin crowing about his plans to take...,1,1,1
38343,10002,"I know one thing , anything that happens , pol...",DISAGREE:Was n't sinjin crowing about his plan...,DISAGREE,FBI Arrests Three Men in Terror Plot that Targ...,Was n't sinjin crowing about his plans to take...,1,1,1
38344,10003,I enjoy Botany more than most things and I hav...,"AGREE:Hi Smallax , welcome to the forum . I di...",AGREE,I enjoy Botany more than most things and I hav...,"Hi Smallax , welcome to the forum . I did a se...",1,1,1


In [ ]:
# train
data = df.loc[df['sub_both'] == 1]

In [ ]:
# train
data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
data['q_end'] = [x+len(y)-1 for x,y in zip(data["q_start"],data["q'"])]
data['r_end'] = [x+len(y)-1 for x,y in zip(data["r_start"],data["r'"])]

<ipython-input-16-851c02192088>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
<ipython-input-16-851c02192088>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
<ipython-input-16-851c02192088>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
data

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1,0,6,76,11
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1,3,6,74,9
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1,0,6,76,9
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1,17,6,61,98
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1,13,6,81,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38341,10001,good thing this argument has never been done !...,"DISAGREE:And teen sex does n't , by the very n...",DISAGREE,You are much better off making theft legal and...,"And teen sex does n't , by the very nature of ...",1,1,1,111,9,227,66
38342,10002,"I know one thing , anything that happens , pol...",DISAGREE:Was n't sinjin crowing about his plan...,DISAGREE,"I know one thing , anything that happens , pol...",Was n't sinjin crowing about his plans to take...,1,1,1,0,9,108,85
38343,10002,"I know one thing , anything that happens , pol...",DISAGREE:Was n't sinjin crowing about his plan...,DISAGREE,FBI Arrests Three Men in Terror Plot that Targ...,Was n't sinjin crowing about his plans to take...,1,1,1,112,9,195,65
38344,10003,I enjoy Botany more than most things and I hav...,"AGREE:Hi Smallax , welcome to the forum . I di...",AGREE,I enjoy Botany more than most things and I hav...,"Hi Smallax , welcome to the forum . I did a se...",1,1,1,0,6,106,125


In [ ]:
# test
df_test

,id,q,r,s
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE
1,5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE
2,8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE
3,1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE
4,6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE
...,...,...,...,...
2011,9499,You are betraying your belief system .,AGREE:Yep . ( I 'm assuming that by `` belief ...,AGREE
2012,4611,"You are in a loud minority , railing against t...",DISAGREE:Being in the minority or in the major...,DISAGREE
2013,9328,You bet your XXX that 'd make me happy .,"DISAGREE:Well , first , I probably would n't b...",DISAGREE
2014,5225,you say `` f * * * the Constitution. ``,DISAGREE:and gun nuts say f * * * the children...,DISAGREE


In [ ]:
train = data

In [ ]:
# test
test = df_test

## Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
train_data_q = train['q'].tolist()
test_data_q = test['q'].tolist()

train_data_r = train['r'].tolist()
test_data_r = test['r'].tolist()

In [ ]:
train_encodings = tokenizer(train_data_q, train_data_r, truncation=True, padding=True)
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)

In [ ]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] It can go both ways. We all doubt. It is what you do with it that matters. [SEP] AGREE : True. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [ ]:
tokenizer.decode(test_encodings['input_ids'][0])

'[CLS] - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ( Time. ) [SEP] DISAGREE : If so, why do we still have apes, and why are there no transitional species that we can currently observe. Statistical averages say we should see a few hundred within the last two centuries, at a minimum. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [ ]:
# train
train_answer = train[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')

In [ ]:
def add_token_positions(encodings, answers):
    q_start, r_start, q_end, r_end = [],[],[],[]

    for i in range(len(answers)):
        q_start.append(encodings.char_to_token(i, answers[i]['q_start'], 0))
        r_start.append(encodings.char_to_token(i, answers[i]['r_start'], 1))
        q_end.append(encodings.char_to_token(i, answers[i]['q_end'], 0))
        r_end.append(encodings.char_to_token(i, answers[i]['r_end'], 1))

        if q_start[-1] is None:
            q_start[-1] = 0
            q_end[-1] = 0
            # continue

        if r_start[-1] is None:
            r_start[-1] = 0
            r_end[-1] = 0
            # continue

        shift = 1
        while q_end[-1] is None:
            q_end[-1] = encodings.char_to_token(i, answers[i]['q_end'] - shift)
            shift += 1
        shift = 1
        while r_end[-1] is None:
            r_end[-1] = encodings.char_to_token(i, answers[i]['r_end'] - shift)
            shift += 1
    encodings.update({'q_start':q_start, 'r_start':r_start,	'q_end':q_end, 'r_end':r_end})

In [ ]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'q_start', 'r_start', 'q_end', 'r_end'])

In [ ]:
test_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# print(train_encodings['q_start'][0])
# print(train_encodings['r_start'][0])
# print(train_encodings['q_end'][0])
# print(train_encodings['r_end'][0])

## Dataset

In [ ]:

class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_dataset = qrDataset(train_encodings)
test_dataset = qrDataset(test_encodings)

In [ ]:
next(iter(train_dataset))

{'input_ids': tensor([  101,  1135,  1169,  1301,  1241,  3242,   119,  1284,  1155,  4095,
           119,  1135,  1110,  1184,  1128,  1202,  1114,  1122,  1115,  5218,
           119,   102, 14731, 16941,  2036,   131,  7817,   119,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
next(iter(test_dataset))

{'input_ids': tensor([  101,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   118,   118,   118,   118,   118,
           118,   118,   118,   118,   118,   113,  2614,   119,   114,   102,
           141,  6258, 22689, 16941,  2036,   131,  1409,  1177,   117,  1725,
          1202,  1195,  1253,  1138,   170,  6633,   117,  1105,  1725,  1132,
          1175,  1185, 23512,  1530,  1115,  1195,  1169,  1971, 12326,   119,
         12121, 22284,  1474,  1195,  1431,  1267,   170,  1374,  2937,  1439,
          1103,  1314,  1160,  3944,   

## Model

In [ ]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 4)


    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out



## Training

In [ ]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-6)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a f

In [ ]:
# Pack data into dataloader by batch
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
training_epoch = 4

In [ ]:
loss_fct = CrossEntropyLoss()

In [ ]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        q_start = batch['q_start'].to(device)
        r_start = batch['r_start'].to(device)
        q_end = batch['q_end'].to(device)
        r_end = batch['r_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_loss = loss_fct(q_start_logits, q_start)
        r_start_loss = loss_fct(r_start_logits, r_start)
        q_end_loss = loss_fct(q_end_logits, q_end)
        r_end_loss = loss_fct(r_end_logits, r_end)



        loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 200 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 200))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0:   5%|▌         | 201/3724 [02:29<44:22,  1.32it/s, loss=21.1]

Epoch 201 Batch 200 Loss 23.1761


Epoch 0:  11%|█         | 401/3724 [05:02<42:30,  1.30it/s, loss=16.9]

Epoch 401 Batch 400 Loss 18.6563


Epoch 0:  16%|█▌        | 601/3724 [07:36<40:09,  1.30it/s, loss=12.5]

...

Epoch 3:  91%|█████████▏| 3401/3724 [43:31<04:07,  1.30it/s, loss=7.46]

Epoch 3401 Batch 3400 Loss 7.2055


Epoch 3:  97%|█████████▋| 3601/3724 [46:05<01:34,  1.30it/s, loss=6.1]

Epoch 3601 Batch 3600 Loss 7.1747


Epoch 3: 100%|██████████| 3724/3724 [47:38<00:00,  1.30it/s, loss=6.77]


In [ ]:
torch.save(model.state_dict(), directory + 'aicup_model_4eplr16.pt')

In [ ]:
model = myModel().to(device)
model.load_state_dict(torch.load(directory + 'aicup_model_4eplr16.pt'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

## Predict

In [ ]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_prdict = torch.argmax(q_start_logits, 1).cpu().numpy()
        r_start_prdict = torch.argmax(r_start_logits, 1).cpu().numpy()
        q_end_prdict = torch.argmax(q_end_logits, 1).cpu().numpy()
        r_end_prdict = torch.argmax(r_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((q_start_prdict[i].item(), r_start_prdict[i].item(), q_end_prdict[i].item(), r_end_prdict[i].item()))

            q_sub = tokenizer.decode(input_ids[i][q_start_prdict[i]:q_end_prdict[i]+1])
            r_sub = tokenizer.decode(input_ids[i][r_start_prdict[i]:r_end_prdict[i]+1])

            q_sub_output.append(q_sub)
            r_sub_output.append(r_sub)

    return q_sub_output, r_sub_output, predict_pos



In [ ]:
q_sub_output, r_sub_output, predict_pos = predict(test_loader)

100%|██████████| 252/252 [01:03<00:00,  3.94it/s]


In [ ]:
def get_output_post_fn(df_test, q_sub_output, r_sub_output):
    q_sub, r_sub = [], []
    for i in range(len(test)):

        q_sub_pred = q_sub_output[i].split()
        r_sub_pred = r_sub_output[i].split()

        if q_sub_pred is None:
            q_sub_pred = []
        q_sub_error_index = q_sub_pred.index('[SEP]') if '[SEP]' in q_sub_pred else -1

        if q_sub_error_index != -1:
            q_sub_pred = q_sub_pred[:q_sub_error_index]

        temp = r_sub_pred.copy()
        if r_sub_pred is None:
            r_sub_pred = []
        else:
            for j in range(len(temp)):
                if temp[j] == '[SEP]':
                    r_sub_pred.remove('[SEP]')
                if temp[j] == '[PAD]':
                    r_sub_pred.remove('[PAD]')

        q_sub.append(" ".join(q_sub_pred))
        r_sub.append(" ".join(r_sub_pred))

    return q_sub, r_sub

In [ ]:
q_sub, r_sub = get_output_post_fn(df_test, q_sub_output, r_sub_output)

In [ ]:
q = []
for i in range(len(q_sub)):
  q_ = "\"" + q_sub[i] + "\""
  q.append(q_)

In [ ]:
r = []
for i in range(len(r_sub)):
  r_ = "\"" + r_sub[i] + "\""
  r.append(r_)

In [ ]:
data = {
    "id": df_test["id"],
    "q": q,
    "r": r
}
df_submit = pd.DataFrame(data)

In [ ]:
df_submit

,id,q,r
0,6199,"""- - - - - - - - - - - - - - - - - - - - - - -...","""If so, why do we still have apes, and why are..."
1,5807,"""There's a lot of discussion there on that iss...","""Of course. The makers of Expelled were within..."
2,8487,"""The guns these people have, they don't regist...","""Oh, I would wager about like Mexico, about 80..."
3,1760,"""Shooting : 3 seriously injured in Auburn shoo...","""Pickup strikes group of four youths | Houston..."
4,6228,"""This is the argument concerning'choice'that, ...","""I believe there is a point at which we ( soci..."
...,...,...,...
2011,9499,"""You are betraying your belief system.""","""Yep. ( I'm assuming that by ` ` belief system..."
2012,4611,"""You are in a loud minority, railing against t...","""Being in the minority or in the majority is i..."
2013,9328,"""You bet your XXX that'd make me happy.""","""Well, first, I probably wouldn't bet my XXX, ..."
2014,5225,"""you say ` ` f * * * the Constitution. ` `""","""and gun nuts say f * * * the children when we..."


In [ ]:
with open(directory + "predict_test.4eplr16.csv", "w", encoding="utf-8") as f:
  df_submit.to_csv(f, index=False)